In [ ]:
import os 
import pandas as pd

# Define the path to the folder
folder_path = r"C:\Users\mydoa\Desktop\BIDFTA DATASET\auctions-dataset\tools\nodejs-dataset-downloader\02_filtered\items" #Defines folder_path, a string with the path to the folder containing files to be processed. The r before the string indicates a raw string to treat backslashes \ as literal characters.
auction_location_path = r"C:\Users\mydoa\Desktop\BIDFTA DATASET\auctions-dataset\tools\nodejs-dataset-downloader\auctions-dataset-filtered-auctions\auctions\auctions.csv"
location_info_path = r"C:\Users\mydoa\Desktop\BIDFTA DATASET\auctions-dataset\tools\nodejs-dataset-downloader\auctions-dataset-filtered-auctions\auctions_data\auctions_locations.csv"
pickupdates_path = r"C:\Users\mydoa\Desktop\BIDFTA DATASET\auctions-dataset\tools\nodejs-dataset-downloader\auctions-dataset-filtered-auctions\auctions_data\auctions_pickupdates.csv"

# Initialize an empty list to store rows of item details directly
data = []

# Load the auctions file containing auction_id to location mapping
try:
    auctions_df = pd.read_csv(auction_location_path)
    print("Loaded auctions file with auction_id to location mapping.")
except Exception as e:
    print(f"An error occurred while loading the auctions file: {e}")


# Create a dictionary mapping auction_id (ID) to location_ID
location_dict = {}
try:
    auctions_df = pd.read_csv(auction_location_path, delimiter='\t', usecols=["ID", "location_ID"])
    auctions_df['ID'] = auctions_df['ID'].astype(str)  # Ensure ID is a string for consistency
    
    # Create a dictionary mapping auction_id (ID) to location_ID
    location_dict = dict(zip(auctions_df['ID'], auctions_df['location_ID']))
    location_dict

except Exception as e:
    print(f"An error occurred while loading the auctions file: {e}")



# Create a dictionary for location details using location_ID as the key
location_info_dict = {}
try:
    location_info_df = pd.read_csv(location_info_path, delimiter='\t', usecols=["id", "state", "zip","tzoffset_utc","tzoffset_et"])
    location_info_df['id'] = location_info_df['id'].astype(str)  # Ensure location_ID (id) is a string
    location_info_df['zip'] = location_info_df['zip'].astype(str)  # Ensure zip is stored as string
    
    # Populate location_info_dict with location_ID as key and (state, zip, tzoffset_utc, tzoffset_et) as values
    location_info_dict = location_info_df.set_index('id')[['state', 'zip']].to_dict(orient='index')
except Exception as e:
    print(f"An error occurred while loading location info file: {e}")




#Create a dictionary for pickupdates as key and auction ID as value
auctionsID_dict = {}
try:
    auctionsID_df = pd.read_csv(pickupdates_path, delimiter='\t', usecols=["auction_ID","date"])
    auctionsID_df['date'] = auctionsID_df['date'].astype(str)
    
    # Populate auctionsID_dict with pickupdates as key and a list of auction_ID as values
    for _, row in auctionsID_df.iterrows():
        auction_id = row['auction_ID']
        pickupdate = row['date']
        if pickupdate in auctionsID_dict:
            auctionsID_dict[pickupdate].append(auction_id)
        else:
            auctionsID_dict[pickupdate] = [auction_id]
    print("Created pickupdates dictionary for pickupdate to auction_ID mapping.")
except Exception as e:
    print(f"An error occurred while loading pickupdates file: {e}")


# Create a dictionary for pickupdates using pickupdates_path
pickupdates_dict = {}
try:
    pickupdates_df = pd.read_csv(pickupdates_path, delimiter='\t', usecols=["auction_ID", "date"])
    pickupdates_df['auction_ID'] = pickupdates_df['auction_ID'].astype(str)  # Ensure auction_ID is a string
    
    # Populate pickupdates_dict with auction_ID as key and a list of pickupdate as values
    for _, row in pickupdates_df.iterrows():
        auction_id = row['auction_ID']
        pickupdate = row['date']
        if auction_id in pickupdates_dict:
            pickupdates_dict[auction_id].append(pickupdate)
        else:
            pickupdates_dict[auction_id] = [pickupdate]
    print("Created pickupdates dictionary for auction_ID to pickupdate mapping.")
except Exception as e:
    print(f"An error occurred while loading pickupdates file: {e}")


# Create a nested dictionary for auction, location and pickupdates
auction_location_pickupdates = {}
try:
    for auction_id, location_id in location_dict.items():
        if auction_id in pickupdates_dict:
            pickupdates = pickupdates_dict[auction_id]
            if auction_id not in auction_location_pickupdates:
                auction_location_pickupdates[auction_id] = {}
            auction_location_pickupdates[auction_id][location_id] = pickupdates
except Exception as e:
    print(f"An error occurred while building the nested dictionary: {e}")


def get_overlapping_auctions(auction_id, auction_location_pickupdates):
    result = []
    location_id = location_dict.get(auction_id)
    pickup_dates = pickupdates_dict.get(auction_id, [])
    for other_auction_id, locations in auction_location_pickupdates.items():
        if location_id in locations:
            if any(date in locations[location_id] for date in pickup_dates):
                result.append(other_auction_id)
    return result


def calculate_items_won_across_auctions(item_id, auction_id, overlapping_auctions):
    overlapping_items = df[df['AUCTION_ID'].isin(overlapping_auctions)]
    
    if overlapping_items.empty:
        print(f"No overlapping items found for Auction: {auction_id}")
        return 0

    current_bidder = df.loc[
        (df['AUCTION_ID'] == auction_id) & (df['ITEM_ID'] == item_id), 'BIDDER_ID'
    ].values
    
    if len(current_bidder) == 0:
        print(f"No BIDDER_ID found for Item: {item_id} in Auction: {auction_id}")
        return 0

    current_bidder = current_bidder[0]
    items_won = overlapping_items[overlapping_items['BIDDER_ID'] == current_bidder].shape[0]
    return items_won
    

def calculate_number_of_winning_overlapping_auctions(row, df):
    if not row['OVERLAPPING']:
        return 0  # If no overlapping auctions, return 0

    overlapping_auctions = row['OVERLAPPING']
    bidder_id = row['BIDDER_ID']

    # Check if the bidder won items in each overlapping auction
    winning_auctions = df[
        (df['AUCTION_ID'].isin(overlapping_auctions)) & 
        (df['BIDDER_ID'] == bidder_id)
    ]['AUCTION_ID'].unique()  # Get unique overlapping auction IDs where the user won items

    return len(winning_auctions)  # Return count of unique winning auctions



# MAIN
try:
    files = os.listdir(folder_path) [:20]  #Uses os.listdir to retrieve a list of all entries (files/folders) in the specified directory.
    print("Processing files in the BIDFTA folder:")

    for file_name in files: #Starts a loop through each file_name in files
        file_path = os.path.join(folder_path, file_name) #Constructs the full path to each file by combining folder_path and file_name
        
        #Checks if file_path is a file (not a folder) with os.path.isfile. If it is a file, prints a message stating which file is currently being processed:
        if os.path.isfile(file_path):
            print(f"\n--- Processing '{file_name}' ---")
            try:
           
                with open(file_path, 'r') as file: #Opens the file in read mode
                    for line_number, line in enumerate(file, start=1): #Loops through each line in the file, with enumerate providing the line number, starting from 1.
                     
                        row_data = line.strip().split('\t') #Strips whitespace from the line and splits it by the tab character ('\t'), storing the resulting list of values in row_data.
                      
                        if line_number==1:
                            continue

                        #Extracts product_id from the fifth element and msrp_value from the twelfth element of row_data.
                        AUCTION_ID = row_data[0]
                        ITEM_ID = row_data[1]  
                        BIDDER_ID = row_data[12] 

                        # Look up location_ID from location_dict
                        LOCATION_ID = location_dict.get(AUCTION_ID, None)

                         # Get overlapping auctions
                        OVERLAPPING = get_overlapping_auctions(AUCTION_ID, auction_location_pickupdates)

                        # Get number of overlapping auctions
                        NUMBER_OF_OVERLAPPING_AUCTIONS = len(OVERLAPPING)

                         # Calculate items won across overlapping auctions
                        WIN_ACROSS_AUCTIONS = calculate_items_won_across_auctions(
                            ITEM_ID, AUCTION_ID, OVERLAPPING
                        )



                        # Append item details
                        data.append([AUCTION_ID, LOCATION_ID, ITEM_ID, BIDDER_ID, OVERLAPPING, NUMBER_OF_OVERLAPPING_AUCTIONS, WIN_ACROSS_AUCTIONS])



            except Exception as e:
                print(f"An error occurred while reading '{file_name}': {e}")

except FileNotFoundError:
    print(f"The folder '{folder_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


# Create DataFrame with headers
df = pd.DataFrame(data, columns=["AUCTION_ID", "LOCATION_ID", "ITEM_ID", "BIDDER_ID", "OVERLAPPING", "NUMBER_OF_OVERLAPPING_AUCTIONS","WIN_ACROSS_AUCTIONS"])


# Ensure user_id is treated as a string and handle empty values
df['BIDDER_ID'] = df['BIDDER_ID'].astype(str).replace('', None)

# Add the new column: "other items in auction"
df['WIN_IN_THIS_AUCTION'] = (
    df[df['BIDDER_ID'].notna()]  # Exclude rows with empty user_id
    .groupby(['AUCTION_ID', 'BIDDER_ID'])['BIDDER_ID']
    .transform('size') 
)

# Fill NaN for rows where user_id is empty with 0
df['WIN_IN_THIS_AUCTION'] = df['WIN_IN_THIS_AUCTION'].fillna(0).astype(int)


df['NUMBER_OF_OVERLAPPING_AUCTIONS_WHERE_THE_USER_WON_ITEMS'] = df.apply(
    calculate_number_of_winning_overlapping_auctions, axis=1, df=df
)


# Define the path for saving the output as CSV
csv_path = r"C:\Users\mydoa\Desktop\BIDFTA DATASET\auctions-dataset\tools\nodejs-dataset-downloader\02_filtered\UPDATED9.csv"
try:
    df.to_csv(csv_path, index=False)  # Save as CSV without the index
    print(f"\nData successfully saved as CSV at '{csv_path}'")
except Exception as e:
    print(f"An error occurred while saving CSV: {e}")